<a href="https://colab.research.google.com/github/tikendraw/Amazon-review-sentiment-analysis/blob/main/amazon-review-sentiment-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Reviews for Sentiment Analysis

## Objective

Here we will be Building ML and DL models to predict the Polarity of reviews.
We will be performing series of experiments with different models to achieve the best classification metrics.(while not abusing the machine we have)

## About Dataset
[Dataset here](https://www.kaggle.com/datasets/kritanjalijain/amazon-reviews)


### OVERVIEW
Contains 34,686,770 Amazon reviews from 6,643,669 users on 2,441,053 products, from the Stanford Network Analysis Project (SNAP). This subset contains 1,800,000 training samples and 200,000 testing samples in each polarity sentiment.

### ORIGIN
The Amazon reviews dataset consists of reviews from amazon. The data span a period of 18 years, including ~35 million reviews up to March 2013. Reviews include product and user information, ratings, and a plaintext review. For more information, please refer to the following paper: J. McAuley and J. Leskovec. Hidden factors and hidden topics: understanding rating dimensions with review text. RecSys, 2013.

### DESCRIPTION
The Amazon reviews polarity dataset is constructed by taking review score 1 and 2 as negative, and 4 and 5 as positive. Samples of score 3 is ignored. In the dataset, class 1 is the negative and class 2 is the positive. Each class has 1,800,000 training samples and 200,000 testing samples.

If you need help extracting the train.csv and test.csv files check out the starter code.

The files train.csv and test.csv contain all the training samples as comma-separated values.

The CSVs contain polarity, title, text. These 3 columns in them, correspond to class index (1 or 2), review title and review text.

polarity - 1 for negative and 2 for positive
title - review heading
text - review body
The review title and text are escaped using double quotes ("), and any internal double quote is escaped by 2 double quotes (""). New lines are escaped by a backslash followed with an "n" character, that is "\n".

In [ ]:
# ! git clone https://github.com/tikendraw/funcyou.git -q
# ! pip install funcyou/
# ! rm -rf funcyou
# ! pip install tensorflow_hub

In [1]:

!pip install contractions

/bin/bash: /home/t/miniconda3/envs/deep/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 1.0 MB/s eta 0:00:00 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 3.5 MB/s eta 0:00:00 MB/s eta 0:00:01


In [2]:
import datetime
import os
import pickle
import random
import re
import sys
import tarfile
import time
from concurrent.futures import ThreadPoolExecutor
from collections import Counter
from pathlib import Path

import contractions
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.utils import check_random_state

from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, TextVectorization
from tensorflow_hub import KerasLayer

try:
    import funcyou
except ImportError:
    # Install funcyou
    !pip install git+https://github.com/tikendraw/funcyou.git@main

# Importing useful functions from funcyou
from funcyou.dataset import download_kaggle_resource
from funcyou.plot import distplot_axis, plot_history, compare_histories
from funcyou.preprocessing.text import IntegerVectorizer
from funcyou.sklearn.metrics import calculate_results

print('Tf version:', tf.__version__)
print('GPU:', len(tf.config.list_physical_devices('GPU')))


2023-09-01 07:49:04.139252: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-01 07:49:05.478811: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Tf version:  2.12.1
GPU:  0


2023-09-01 07:49:08.897788: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-01 07:49:09.060684: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
def set_global_seed(seed):
    tf.random.set_seed(seed)
    global random_state
    random_state = check_random_state(seed)


# Set the seed value
SEED = 42 # should be same as seed in config file

# Call the function to set the seeds
set_global_seed(SEED)


In [4]:
# download and enter the repo to access other files

colab = 'google.colab' in str(get_ipython())

if colab:
    os.system('git clone https://github.com/tikendraw/Amazon-review-sentiment-analysis.git')

    if os.getcwd() != '/content/Amazon-review-sentiment-analysis/notebook':
        os.chdir('Amazon-review-sentiment-analysis/notebook')
    print('Current working directory: ',os.getcwd())
    
    
    # load google drive 
    from google.colab import drive
    drive.mount('/content/drive')


In [6]:
cur_dir = Path(os.getcwd())
model_dir = Path(os.getcwd()).parent /'model'
data_dir = Path(os.getcwd()).parent /'dataset'


In [7]:
print(cur_dir)
print(model_dir)
print(data_dir)

/home/t/aproject/Amazon-review-sentiment-analysis/notebook
/home/t/aproject/Amazon-review-sentiment-analysis/model
/home/t/aproject/Amazon-review-sentiment-analysis/dataset


In [14]:
# Download the data if you don't have locally
dataset_url = 'https://www.kaggle.com/datasets/kritanjalijain/amazon-reviews'

In [15]:
if colab:
    download_kaggle_resource(dataset_url, data_dir, kaggle_json_path = '/content/kaggle.json')

    from zipfile import ZipFile
    with ZipFile(data_dir/'amazon-reviews'/'amazon-reviews.zip') as f:
        f.extractall(data_dir)

# Load the data

In [16]:
#reading data
df = pl.read_csv(data_dir/'train.csv',new_columns = ['polarity', 'title','text'])  # gives TextFileReader, which is iterable with chunks of 1000 rows.


In [17]:
df.describe()

describe,polarity,title,text
str,f64,str,str
"""count""",3.599999e6,"""3599999""","""3599999"""
"""null_count""",0.0,"""0""","""0"""
"""mean""",1.5,null,null
"""std""",0.5,null,null
"""min""",1.0,"""""","""this is the b…"
"""max""",2.0,"""♦ LOVE IT ♦""","""…were Marvin a…"
"""median""",1.0,null,null
"""25%""",1.0,null,null
"""75%""",2.0,null,null


In [18]:
# check for nulls and drop if any
df.null_count()

polarity,title,text
u32,u32,u32
0,0,0


In [19]:
#drop nulls
df.drop_nulls()
print()

In [20]:
# checking for classs imbalance
df['polarity'].value_counts()

polarity,counts
i64,u32
1,1800000
2,1799999


# We will map the polarity between 0 for negative sentiment to 1 for positive sentiment

In [21]:
df = df.with_columns([
                    pl.col('polarity').apply(lambda x: 0 if x == 1 else 1).alias('polarity')
                     ])

df = df.with_columns([
                     pl.col('polarity').cast(pl.Int16, strict=False).alias('polarity')
                     ])


In [22]:
df.sample(10)

polarity,title,text
i16,str,str
1,"""A Delightful S…","""Although I hav…"
0,"""Disaster""","""This system is…"
0,"""Very poor book…","""This book offe…"
0,"""Amanda has sho…","""I bought this …"
0,"""Blood and guts…","""If you liked ""…"
1,"""Captures the E…","""Heartfelt roma…"
0,"""All my Beads j…","""OMG! This coul…"
0,"""Crappy Program…","""Apart from the…"
1,"""Why, oh, why..…","""...Do these gu…"


## Note: We will be combining text and title columns . makes more sense.

In [23]:
df = df.with_columns([
    (pl.col('title')+' ' + pl.col('text')).alias('review')
])

df = df.with_columns([
    pl.col('review').str.to_lowercase()
])


In [24]:
# df = df.with_columns([
#     pl.col('title').apply(lambda x: len(str(x).split())).alias('title_len'),
# ])

# df = df.with_columns([
#     pl.col('text').apply(lambda x: len(str(x).split())).alias('text_len'),
# ])

# df = df.with_columns([
#     pl.col('review').apply(lambda x: len(str(x).split())).alias('review_len')
# ])


In [26]:

# plt.figure(figsize=(15,5))
# distplot_axis(df['title_len'].to_numpy())

# plt.figure(figsize=(15,5))
# distplot_axis(df['text_len'].to_numpy())

# plt.figure(figsize=(15,5))
# distplot_axis(df['review_len'].to_numpy())

In [27]:
df = df.select(['review', 'polarity'])

In [28]:
df.head()

review,polarity
str,i16
"""the best sound…",1
"""amazing! this …",1
"""excellent soun…",1
"""remember, pull…",1
"""an absolute ma…",1


In [44]:
df.write_csv(data_dir / 'preprocessed_df.csv')

In [45]:
df = pl.read_csv(data_dir/'preprocessed_df.csv')
df.head()

review,polarity
str,i64
"""the best sound…",1
"""amazing! this …",1
"""excellent soun…",1
"""remember, pull…",1
"""an absolute ma…",1


## Preprocessing text

In [48]:

# Compile the regular expressions outside the function for better performance
PUNCTUATION_REGEX = re.compile(r'[^\w\s]')
DIGIT_REGEX = re.compile(r'\d')
SPECIAL_CHARACTERS_REGEX = re.compile(r'[#,@,&]')
MULTIPLE_SPACES_REGEX = re.compile(r'\s+')

def clean_text(x: str) -> str:
    expanded_text = contractions.fix(x)  # Expand contractions
    text = PUNCTUATION_REGEX.sub(' ', expanded_text.lower())  # Remove punctuation after lowering
    text = DIGIT_REGEX.sub('', text)  # Remove digits
    # Remove special characters (#,@,&)
    text = SPECIAL_CHARACTERS_REGEX.sub('', text)
    # Remove multiple spaces with single space
    text = MULTIPLE_SPACES_REGEX.sub(' ', text)
    return text.strip()

# Compile regex patterns for better performance


In [47]:
q = "i'm going to market.you wanna come?huh?"
clean_text(q)

'i am going to market you want to come huh'

In [29]:
counter_object_filepath = model_dir / 'counter.pkl'

save_dir = Path('/content/drive/MyDrive/amazom_sentiment')
if colab:
    counter_object_filepath = save_dir / 'counter.pkl'


In [50]:
# a = IntegerVectorizer(preprocessing_func=clean_text, min_freq=3) # 19 min  , contraction fix takes most of the time
# a.adapt(df['review'])


Vocab size: 261828


In [54]:
# # save the counter object
# counter_object = a.vocab.counter  #Vocab size: 261933

# with open(counter_object_filepath, 'wb') as output_file:
#     pickle.dump(counter_object, output_file)
#     print('file saved at',counter_object_filepath)

file saved at /home/t/aproject/Amazon-review-sentiment-analysis/model/counter.pkl


In [33]:
with open(counter_object_filepath, 'rb') as output_file:
    counter_object = pickle.load(output_file)

In [62]:
# Get the least common words
least_common_words = counter_object.most_common()[::-1]


In [67]:
f = 100_000
counter_object.most_common()[f:f+100]

[('eclair', 14),
 ('krown', 14),
 ('pti', 14),
 ('winterbourne', 14),
 ('stracher', 14),
 ('zekk', 14),
 ('slection', 14),
 ('fiv', 14),
 ('kreh', 14),
 ('kaneda', 14),
 ('xhr', 14),
 ('funning', 14),
 ('guenivere', 14),
 ('aiki', 14),
 ('erring', 14),
 ('sundiver', 14),
 ('retirements', 14),
 ('carryin', 14),
 ('wfp', 14),
 ('lunt', 14),
 ('hydrangeas', 14),
 ('coombe', 14),
 ('precisa', 14),
 ('reeler', 14),
 ('sesson', 14),
 ('handsomest', 14),
 ('milimeter', 14),
 ('gastar', 14),
 ('suggestible', 14),
 ('splitted', 14),
 ('koboi', 14),
 ('kemprecos', 14),
 ('shoppin', 14),
 ('axillary', 14),
 ('brawlers', 14),
 ('whodunnits', 14),
 ('anquish', 14),
 ('convened', 14),
 ('alsoa', 14),
 ('unopen', 14),
 ('openner', 14),
 ('jomini', 14),
 ('referb', 14),
 ('gimmee', 14),
 ('inteligencia', 14),
 ('fcs', 14),
 ('gumy', 14),
 ('corrals', 14),
 ('baleful', 14),
 ('procedings', 14),
 ('chipettes', 14),
 ('highliter', 14),
 ('perfers', 14),
 ('incrdible', 14),
 ('tullock', 14),
 ('stofen', 1

In [68]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1], reverse = False)[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                counter_object: 29.3 MiB
            least_common_words:  6.8 MiB
                 MultinomialNB:  1.6 KiB
               TfidfVectorizer:  1.6 KiB
                      Pipeline:  1.6 KiB
                     Embedding:  1.6 KiB
                          LSTM:  1.6 KiB
                         Dense:  1.6 KiB
                       Dropout:  1.6 KiB
             TextVectorization:  1.6 KiB


# Text vectorization

In [69]:
MAX_TOKEN = 100_000
OUTPUT_SEQUENCE_LENGTH = 175  # limiting reviews to 175 words

creating a dictionary of words with counts helps me create text vectorizer alot faster .


it only took 3min 24sec to create the dictionary from 3.6 million entries which is alot faster if you try to
adapt  TextVectorization to the data.

In [71]:
text_vectorizer = TextVectorization(max_tokens=MAX_TOKEN, standardize='lower_and_strip_punctuation',
                                   split='whitespace',
                                    ngrams= None ,
                                    output_mode='int',
                                    output_sequence_length=OUTPUT_SEQUENCE_LENGTH,
                                    pad_to_max_tokens=True,
                                    vocabulary = list(counter_object.keys())[:MAX_TOKEN-2]) # -2 FOR 'PAD' AND 'UNK' TOKEN

In [72]:
random_review = "WHo the duck are you? aren't you a goose?"

In [73]:
word_to_id = tf.keras.layers.StringLookup(vocabulary=text_vectorizer.get_vocabulary(), mask_token='', oov_token='[UNK]')
id_to_word = tf.keras.layers.StringLookup(vocabulary=text_vectorizer.get_vocabulary(), mask_token='', oov_token='[UNK]', invert=True)

In [74]:
clean_text(random_review)

'who the duck are you are not you a goose'

In [79]:
print('random Review: ', random_review)
print('random Review length: ', len(random_review))
print('-------\n')
print('vectorized review: ',text_vectorizer([random_review]))

print('Reverse vectorized review: ',id_to_word(text_vectorizer([random_review])))


print('-------\n')
print('Vocabulary_length: ',len(text_vectorizer.get_vocabulary()))
print('Most frequent words: ',text_vectorizer.get_vocabulary()[:100])
print('least frequent words: ',text_vectorizer.get_vocabulary()[-10:])

random Review:  WHo the duck are you? aren't you a goose?
random Review length:  41
-------

vectorized review:  tf.Tensor(
[[   91     2  8076    58    57     1    57    11 30011     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0 

# Data Preparation

In [ ]:

TEST_SIZE = .01 # same as config.TEST_SIZE

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split( df.select('review'), df.select('polarity'), test_size=  .01,  random_state = SEED)

In [ ]:
del(df)

In [ ]:
print('xtrain shape',xtrain.shape, 'ytrain shape', ytrain.shape)
print('xtest shape',xtest.shape, 'ytest shape', ytest.shape)

xtrain shape (3563999, 1) ytrain shape (3563999, 1)
xtest shape (36000, 1) ytest shape (36000, 1)


# Creating tensorflow dataset using `tf.data` api

In [ ]:
BATCH_SIZE = 32

In [ ]:
def data_generator(x, y):
    num_samples = len(x)
    for i in range(num_samples):
        yield x[i], y[i]


def create_datasets(x, y, text_vectorizer, batch_size, shuffle=True, buffer_size=10000):

    generator = data_generator(x, y)
    print('Generating...')
    train_dataset = tf.data.Dataset.from_generator(
        lambda: generator,
        output_signature=(
            tf.TensorSpec(shape=(None, x.shape[1]), dtype=tf.string),
            tf.TensorSpec(shape=(None, y.shape[1]), dtype=tf.int32)
        )
    )
    print('Mapping...')
    train_dataset = train_dataset.map(lambda x, y: (tf.cast(text_vectorizer(x), tf.int32)[0], y[0]), tf.data.AUTOTUNE)
    train_dataset = train_dataset.batch(batch_size)

    if shuffle:
        train_dataset = train_dataset.shuffle(buffer_size)
    train_dataset = train_dataset.cache().repeat().prefetch(tf.data.AUTOTUNE)
    print('Done.')
    return train_dataset


In [ ]:
# ls

In [ ]:
train_dataset = create_datasets(xtrain, ytrain, text_vectorizer, batch_size=BATCH_SIZE, shuffle=False)
test_dataset = create_datasets(xtest, ytest, text_vectorizer, batch_size=BATCH_SIZE, shuffle=False)

Generating...
Mapping...
Done.
Generating...
Mapping...
Done.


In [ ]:
# del(xtrain, ytrain)
# del(xtest, ytest, counter_object)

In [ ]:
# train_features = tf.data.Dataset.from_tensor_slices(xtrain)
# train_label = tf.data.Dataset.from_tensor_slices(ytrain)
# test_features = tf.data.Dataset.from_tensor_slices(xtest)
# test_label = tf.data.Dataset.from_tensor_slices(ytest)

# del(xtrain, ytrain, xtest, ytest) # deleting variables to free the memory

In [ ]:
# train_dataset = tf.data.Dataset.zip((train_features, train_label))
# train_dataset = train_dataset.map(lambda x,y: (text_vectorizer(x)[0],y),tf.data.AUTOTUNE)
# train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# test_dataset = tf.data.Dataset.zip((test_features, test_label))
# test_dataset = test_dataset.map(lambda x,y: (text_vectorizer(x)[0],y),tf.data.AUTOTUNE )
# test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# del(train_features, train_label, test_features, test_label) # deleting variables to free the memory

In [ ]:
# print('len train dataset: ', len(train_dataset)) # doesn't work for generator generated tensor data
# print('len test dataset: ', len(test_dataset))  # doesn't work for generator generated tensor data


In [ ]:
for i,j in train_dataset:
    print(i.shape,j.shape)
    print(i,j)
    break

(32, 175) (32, 1)
tf.Tensor(
[[ 9267    43  1173 ...     0     0     0]
 [   11   397   204 ...     0     0     0]
 [  236   224   417 ...     0     0     0]
 ...
 [ 1051    20    43 ...     0     0     0]
 [  415 19030     8 ...     0     0     0]
 [    1  1449    76 ...     0     0     0]], shape=(32, 175), dtype=int32) tf.Tensor(
[[1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]], shape=(32, 1), dtype=int32)


In [ ]:
train_length = xtrain.shape[0]// BATCH_SIZE
test_length = xtest.shape[0] // BATCH_SIZE

In [ ]:
print("train length: ", train_length, '\ntest_length: ', test_length)

train length:  111374 
test_length:  1125


In [ ]:
VECTORIZER_PATH = model_dir / 'text_vectorizer.pkl'
VECTORIZER_PATH

PosixPath('/content/Amazon-review-sentiment-analysis/model/text_vectorizer.pkl')

In [ ]:
# to load text vectorizer
def load_text_vectorizer(vectorizer_path):
    from_disk = pickle.load(open(vectorizer_path, "rb"))
    return TextVectorization.from_config(from_disk['config'])

# Pickle the config and weights
def save_text_vectorizer(vectorizer_path):
    pickle.dump({'config': text_vectorizer.get_config()}
                , open(vectorizer_path, "wb"))


In [ ]:
text_vectorizer.vocabulary_size(), text_vectorizer.get_vocabulary()[:4]

(300000, ['', '[UNK]', 'the', 'best'])

In [ ]:
# text_vectorizer = load_text_vectorizer(model_dir/'text_vectorizer.pkl')

In [ ]:
ls -la -h ../model

total 56M
drwxr-xr-x 2 root root 4.0K Aug 31 10:55 ./
drwxr-xr-x 7 root root 4.0K Aug 31 10:46 ../
-rw-r--r-- 1 root root  12M Aug 31 11:20 counter.pkl
-rw-r--r-- 1 root root  44M Aug 31 10:09 text_vectorizer.pkl


In [ ]:
save_text_vectorizer(VECTORIZER_PATH)

# Embedding

In [ ]:
DIM = 8

In [ ]:
embedding = Embedding(input_dim = MAX_TOKEN,output_dim= DIM, mask_zero=True, input_length=OUTPUT_SEQUENCE_LENGTH)
print('Embedded text vectorized random sentence: ',embedding(text_vectorizer(random_review)))

Embedded text vectorized random sentence:  tf.Tensor(
[[-0.02809688  0.03082368 -0.02951168 ... -0.01749847  0.00370653
  -0.00014732]
 [ 0.00214576 -0.00436879 -0.04995276 ... -0.01645011 -0.03323325
  -0.02844541]
 [ 0.04596445  0.03137359  0.04264681 ...  0.04373202  0.0074513
  -0.03073972]
 ...
 [ 0.03726716 -0.01544239  0.02323947 ...  0.02113296  0.03661327
  -0.0458019 ]
 [ 0.03726716 -0.01544239  0.02323947 ...  0.02113296  0.03661327
  -0.0458019 ]
 [ 0.03726716 -0.01544239  0.02323947 ...  0.02113296  0.03661327
  -0.0458019 ]], shape=(175, 8), dtype=float32)


# Model:0 (Naive bayes model)

In [ ]:
# model0 = Pipeline([
#     ('tfidf',TfidfVectorizer()),
#     ('multino',MultinomialNB())
# ])

In [ ]:
# %%time
# #fit and predict
# model0.fit(xtrain['review'], ytrain)

In [ ]:
# pred0 = model0.predict(xtest['review'])

# print(pred0.shape ==  ytest.shape)
# print('pred00.shape: ',pred0.shape)
# print('ytest.shape: ',ytest.shape)

# model0_res = calculate_results(y_true=ytest, y_pred=pred0, model_name='model0: naive bayes')
# print(model0_res)

{'model': 'model0: naive bayes', 'accuracy': 84.79444444444444, 'precision': 0.8482905354316962, 'recall': 0.8479444444444444, 'f1': 0.8479237632137091, 'discription': None}

# Model1

In [ ]:

def create_lstm_model(input_shape, max_tokens, dim):
    inputs = keras.Input(shape=(input_shape))
    embedding_layer = Embedding(input_dim=max_tokens, output_dim=dim, mask_zero=True, input_length=input_shape, name='embedding_layer')(inputs)
    x = LSTM(16, return_sequences=True, name = 'lstm_layer_1')(embedding_layer)
    x = LSTM(16, name = 'lstm_layer_2')(x)
    x = Dropout(0.4, name ='dropout_layer')(x)
    x = Dense(64, activation='relu', name = 'dense_layer_1')(x)
    outputs = Dense(1, activation='sigmoid', name = 'dense_layer_2_final')(x)
    return keras.Model(inputs=inputs, outputs=outputs, name='model_lstm')



In [ ]:
import tensorflow as tf

class LSTMModel(tf.keras.Model):
    def __init__(self, text_vectorizer, embed_dim, dropout_rate=0.2):
        super(LSTMModel, self).__init__(name='custom_lstm_model')
        self.text_vectorizer = text_vectorizer

        self.embedding_layer = tf.keras.layers.Embedding(input_dim=text_vectorizer.vocabulary_size(), output_dim=embed_dim)
        self.lstm1 = tf.keras.layers.LSTM(16, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(16)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.embedding_layer(inputs)
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.dropout(x)
        x = self.dense1(x)
        return self.dense2(x)


    def predict_text(self, text):
        vectorized_text = self.text_vectorizer.transform([text])
        return self.predict(vectorized_text)


# Instantiate the model
output_sequence_length = OUTPUT_SEQUENCE_LENGTH  # actual value
# model = LSTMModel(text_vectorizer, embed_dim=DIM)
model = create_lstm_model(output_sequence_length, max_tokens=MAX_TOKEN, dim=DIM)

# Compile the model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['Accuracy'])

In [ ]:
model.summary()

Model: "model_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 175)]             0         
                                                                 
 embedding_layer (Embedding)  (None, 175, 8)           2400000   
                                                                 
 lstm_layer_1 (LSTM)         (None, 175, 16)           1600      
                                                                 
 lstm_layer_2 (LSTM)         (None, 16)                2112      
                                                                 
 dropout_layer (Dropout)     (None, 16)                0         
                                                                 
 dense_layer_1 (Dense)       (None, 64)                1088      
                                                                 
 dense_layer_2_final (Dense)  (None, 1)                6

In [ ]:
# tf.keras.utils.plot_model(model, expand_nested=True, show_shapes=True, show_dtype=True, rankdir='TP')

In [ ]:
model_dir = Path(os.getcwd()).parent /'model'
model_dir

PosixPath('/content/Amazon-review-sentiment-analysis/model')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
# folder path to save the model weights
drive_filepath = "/content/drive/My Drive/amazom_sentiment"

# Define the ModelCheckpoint callback
checkpoint_filepath = os.path.join(drive_filepath, "model_weights.h5")
model_checkpoint_callback = ModelCheckpoint(checkpoint_filepath, monitor = 'loss', save_best_only=True, save_weights_only=True)


In [ ]:
# model = tf.keras.models.load_model(model_dir/'model.h5')
model.load_weights(checkpoint_filepath)

In [ ]:
EPOCHS = 20
steps_per_epoch = int(1.0 * train_length//EPOCHS)
validation_steps = int(1.0 * test_length//EPOCHS)

print('Epochs          : ', EPOCHS)
print('train_length    :',train_length )
print('test_length     :   ',test_length )
print('steps_per_epoch : ',steps_per_epoch)
print('validation_steps: ',validation_steps)

Epochs          :  20
train_length    : 111374
test_length     :    1125
steps_per_epoch :  5568
validation_steps:  56


In [ ]:
# %%time

# fit the model
history1 = model.fit(train_dataset, epochs = EPOCHS,
                      validation_data= test_dataset,
                      steps_per_epoch = steps_per_epoch,
                      validation_steps=validation_steps,
                     callbacks=[model_checkpoint_callback]
                    )

Epoch 1/20
5568/5568 [==============================] - ETA: 0s - loss: 0.1717 - Accuracy: 0.9345

5568/5568 [==============================] - 212s 38ms/step - loss: 0.1717 - Accuracy: 0.9345 - val_loss: 0.3289 - val_Accuracy: 1.0000
Epoch 2/20
5568/5568 [==============================] - 212s 38ms/step - loss: 0.1654 - Accuracy: 0.9376
Epoch 3/20
5568/5568 [==============================] - 213s 38ms/step - loss: 0.1603 - Accuracy: 0.9392
Epoch 4/20
5568/5568 [==============================] - 212s 38ms/step - loss: 0.1580 - Accuracy: 0.9399
Epoch 5/20
5568/5568 [==============================] - 212s 38ms/step - loss: 0.1544 - Accuracy: 0.9418
Epoch 6/20
5568/5568 [==============================] - 209s 37ms/step - loss: 0.1529 - Accuracy: 0.9427
Epoch 7/20
5568/5568 [==============================] - 208s 37ms/step - loss: 0.1524 - Accuracy: 0.9429
Epoch 8/20
5568/5568 [==============================] - 210s 38ms/step - loss: 0.1492 - Accuracy: 0.9434
Epoch 9/20
5568/5568 [==============================] - 211s 38ms/step - loss: 0.1487 - Accuracy: 0.9434
Epoch 10/20
5568/5568 [=

5568/5568 [==============================] - 0s 36us/step - loss: 0.0617 - Accuracy: 0.9837


In [ ]:
# model_dir

In [ ]:
model.save('/content/drive/MyDrive/amazom_sentiment/full_model.h5')
# model.load_weights(checkpoint_filepath)

In [ ]:
from funcyou.plot import plot_history

#### Evaluation

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 175), dtype=tf.int32, name=None), TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))>

In [ ]:
ypred = tf.squeeze(tf.round(model.predict(test_dataset)))
print('ypred1.shape: ',ypred.shape)

# model1_res = calculate_results(ytest,ypred, model_name='model1: LSTM', discription = 'small lstm model with vectorizer and embedding layer')
# print(model1_res)


1/1 [==============================] - 3s 3s/step
ypred1.shape:  (3,)


ValueError: ignored

In [ ]:

def predict_sentiment(title, text, text_vectorizer, lstm_model):
    review = f'{title} {text}' # concatenate the title and text
    clean_review = clean_text(review)
    review_sequence = text_vectorizer([clean_review])
    prediction = lstm_model.predict(review_sequence)
    sentiment_score = prediction[0][0]
    sentiment_label = 'Positive' if sentiment_score >= 0.5 else 'Negative'
    return sentiment_label, sentiment_score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 6.2 MB/s eta 0:00:00


In [ ]:
a = "i'm don't , y'll, won't"
clean_text(a)

'i am do not   y ll  will not'

In [ ]:
review_title = 'my name is mike'
review_text  =  "and i don't like it"

sentiment_label, sentiment_score = predict_sentiment(review_title, review_text, text_vectorizer, model)
print(sentiment_label, sentiment_score)

1/1 [==============================] - 0s 406ms/step
Negative 0.21370934
